In [1]:
] activate

 Activating environment at `~/Dropbox/EMAC/DCSideBatteryModeling/Project.toml`


In [2]:
using LinearAlgebra

using OrdinaryDiffEq #Gets the solvers
using Plots
using PowerSystems
using DelimitedFiles

include(joinpath(pwd(), "DCSideBatteryModeling", "DCSideBatteryModeling.jl"))

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260
┌ Info: Precompiling PowerSystems [bcd98974-b02a-5e2f-9ee0-a103f5c450dd]
└ @ Base loading.jl:1260


In [3]:
plotly()

┌ Info: Precompiling ORCA [47be7bcc-f1a6-5447-8b36-7eeeff7534fd]
└ @ Base loading.jl:1260
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for ORCA [47be7bcc-f1a6-5447-8b36-7eeeff7534fd]
│   exception = ErrorException("Required dependency PlotlyBase [a03496cd-edff-5a9b-9e67-9cda94a718b5] failed to load from a cache file.")
└ @ Base loading.jl:1041


Plots.PlotlyBackend()

In [ ]:
include(joinpath(pwd(), "data","make_data.jl"))

[8383:0809/124158.009308:ERROR:buffer_manager.cc(488)] [.DisplayCompositor]GL ERROR :GL_INVALID_OPERATION : glBufferData: <- error from previous GL command
┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/5TSyV/src/parsers/pm_io/common.jl:26
┌ Info: Constructing System from Power Models
│   data["name"] = omib
│   data["source_type"] = pti
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/5TSyV/src/parsers/power_models_data.jl:39
┌ Info: Created InMemoryTimeSeriesStorage
└ @ InfrastructureSystems /home/ciaran/.julia/packages/InfrastructureSystems/uXGZ2/src/in_memory_time_series_storage.jl:24
┌ Info: Reading bus data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/5TSyV/src/parsers/power_models_data.jl:140


In [ ]:
omib_sys = System(joinpath(pwd(), "data", "OMIB_inverterDCside.json"))

## Instantiate 0th-order ODE Model

In [ ]:
parameter_mapping = instantiate_parameters(omib_sys)
M= instantiate_0th_order_model(omib_sys)

In [ ]:
_parameter_values = [x.second for x in parameter_mapping] # Extract list of parameter values

## Build Jacobian Function

In [ ]:
jac_exp = get_0th_order_jacobian_expression()
_jac = eval(jac_exp)

In [ ]:
jac_eval = (out, u0, params) -> _jac(out, u0, params)
param_eval = (out, params) -> _jac(out, M.u0, params)
n = length(M.u0)
J = zeros(n, n);

In [ ]:
M(_parameter_values)
param_eval(J, _parameter_values)
jac = ModelJacobian(jac_eval, J)

In [ ]:
parameter_mapping[45]

## Identfying Set of Small-Signal Stable Gains

In [ ]:
cap_range = 2000:500:7000 # Looping throught different DC-link capacitor sizing
param_range = 0.5:0.5:10.5 # Looping throughc control  
_parameter_values[45]=0 # Setting K_pred=0
max_eig = -3 # Design specification - Maximum real part of eigenvalues
min_damping = 0.35 # Design specification - Minimum damping
for cap_size in cap_range
    _parameter_values[39]=cap_size*1e-6/_parameter_values[2] # Assigning DC-link capacitor size
    stable_gains = reshape([],0,4) # Initializing array of stable gains
    for kpv in param_range
        _parameter_values[41]=kpv # K_p^v
        for kiv in param_range  
            _parameter_values[42]=kiv #K_i^v
            for kpi in param_range
                _parameter_values[43]=kpi #K_p^i
                for kii in param_range
                    _parameter_values[44]=kii #K_i^i
                    M(_parameter_values) # Re-calculating operating point 
                    param_eval(J, _parameter_values) 
                    jac = ModelJacobian(jac_eval, J)
                    eig_vals = eigvals(jac(M)) # Calculating eigenvalues of linearized system
                    damp = [-1*real(λ) / sqrt(real(λ)^2 + imag(λ)^2) for λ in eig_vals] # Calcualting damping ratios
                    if maximum(real(eig_vals)) < max_eig && minimum(damp)>min_damping # Checking to see if gains satisfy design criteria
                        stable_gains = [stable_gains; [kpv kiv kpi kii ]] # Concatenating stable gains
                    end
                end 

            end

        end

    end
    fileName=string("results/stable_gains/capSize_", cap_size, "uF.csv")
    writedlm( fileName,  stable_gains, ',') # Writing array of gains that satisfy design criteria 
end

## Searching Over Set of Stable Gains for Large Signal Tuning

In [ ]:
cap_range = 2000:500:7000 # Looping throught different DC-link capacitor sizing
_parameter_values[45]=0 #K_pred
for cap_size in cap_range
    _parameter_values[39]=cap_size*1e-6/_parameter_values[2] # Assigning DC-link capacitor size
    fileName=string("results/stable_gains/capSize_", cap_size, "uF.csv")
    gains = readdlm(fileName, ',', Float64) # Loading array of Stable Gains
#     fileName=string("results/optimal_gains/capSize_", cap_size, "uF.csv")
#     optIndex = readdlm(fileName, ',', Float64)
    optIndex = [1, 1e6] # Initializing optIndex (index 1) and associated l_2 norm values (index 2)
    for i in 1:1:1000
        index = rand(1:size(gains,1))
        _parameter_values[41]=gains[index,1] #K_p^v
        _parameter_values[42]=gains[index,2] #K_i^v
        _parameter_values[43]=gains[index,3] #K_p^i
        _parameter_values[44]=gains[index,4] #K_i^i
        M(_parameter_values) #Re-calcualte operating point

        disturbance=deepcopy(_parameter_values)
        disturbance[10]=0.7 # Changing active power load from 0.2 to 0.7 p.u.

        sim = instantiate_0th_order_dae(disturbance, M; tspan = (0.0, 0.02))
        sol = solve(sim, Rodas5(), saveat = 0.001);
        solArray = Array(sol)
        global runError=norm(_parameter_values[40].-solArray[13,:]) # Calculate l2 norm error
        if runError < optIndex[2] # If l_2 norm error is less that previously stored
            optIndex[1]=index  # Store current array index as optimal index
            optIndex[2]=runError
            fileName=string("results/optimal_gains/capSize_", cap_size, "uF.csv")
            writedlm(fileName,  optIndex, ',') #Write csv with index and l_2 norm error
        end
    end
end